In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
import nltk
import  sklearn
from collections import Counter
stemmer = SnowballStemmer('english')
news = pd.read_csv('/home/linu/news_articles.csv')
news.head()
tokenizer = ToktokTokenizer()
stops = set(stopwords.words('english'))
from sklearn.decomposition import LatentDirichletAllocation

no_of_recommends = 20
n_topics = 10
no_of_clusters = 10
news = news[['Article_Id','Title','Content']].dropna()
contents = news["Content"].tolist()
title = news['Title']
article_id = news['Article_Id']
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.preprocessing import normalize
import pickle as pk

In [2]:
def clean_tokenize(document):
    document = re.sub('[^\w_\s-]',' ',document)
    tokens  = nltk.word_tokenize(document)
    cleaned_article = ' '.join([stemmer.stem(item) for item in tokens])   #stemming the tokenized corpus
    return cleaned_article

cleaned_articles = list(map(clean_tokenize,contents))

In [3]:

def Topic_Modeller(LDA_matrix):
    
    total_WordVocab = []
    for i in range(0,len(cleaned_articles)) :
        word_tokens = nltk.word_tokenize(cleaned_articles[i])
        for words in word_tokens :
            total_WordVocab.append(words)
        counts = Counter(total_WordVocab)

    vocab = {j:i for i,j in enumerate(counts.keys())}

    stops_removed = [word for word in vocab.keys() if word not in stops]

    Final_VocabDict = {j:i for i,j in enumerate(stops_removed)}
    
    Tfidf = TfidfVectorizer(min_df=1,vocabulary=Final_VocabDict)


    Tfidf_Matrix = Tfidf.fit_transform(cleaned_articles)

    Lda = LatentDirichletAllocation(n_components=n_topics,max_iter=1,random_state=0)
    
    Lda_articlemat = Lda.fit_transform(Tfidf_Matrix)

    return Lda_articlemat  

In [4]:
wordtokens_article = [word.split() for word in cleaned_articles]  #for userprofiles 

In [5]:
# with open('topic_modeled.pickle','wb') as tm :
#     pk.dump(Lda_articlemat,tm) 

In [6]:
#Lda_articlemat = Topic_Modeller(cleaned_articles)

In [7]:
with open('topic_modeled.pickle','rb') as topics :
    
    Lda_articlemat = pk.load(topics)

In [8]:
kmeans = KMeans(n_clusters=no_of_clusters)

In [15]:
clustered_articles_matrix = kmeans.fit_transform(Lda_articlemat)

In [16]:
clustered_articles_matrix.shape

(4831, 10)

In [17]:
wordtokens_article = [word.split() for word in cleaned_articles]

In [18]:
Lda_articlemat[500]

array([0.01256278, 0.34295554, 0.01256291, 0.01256288, 0.5565409 ,
       0.01256336, 0.01256272, 0.0125628 , 0.01256307, 0.01256303])

In [19]:
clustered_articles_matrix[500]

array([1.03596345, 1.00621297, 1.0260601 , 1.03605366, 1.01270339,
       1.0336135 , 1.03342534, 1.01845823, 0.39285391, 0.73123103])

In [20]:
def user_profiler(wordtokens,article_read,article_time):
    user_profile = []
    wordPer_second = 5
    

    for i in range(len(wordtokens)):                                        

        average_time = (len(wordtokens[i])/wordPer_second) #length of wordtokslist by wps gives us avg time to read the article
         
        user_interest_timevalue = article_time[i]/average_time  #article_times divide by avg times of each article                   
        
        user_profile_generate = (article_read[i]*user_interest_timevalue)   #clustered_articles_matrix[] * user_interest_time calculated                 
        
        user_profile.append(user_profile_generate)                                      

    return sum(normalize(user_profile))                             


userProfile_one = user_profiler([wordtokens_article[600],wordtokens_article[99],wordtokens_article[120]],
                             [clustered_articles_matrix[600],clustered_articles_matrix[99],clustered_articles_matrix[120]],
                             [120,60,30])


In [ ]:
userProfile_one

In [21]:
def Content_Recommends_Calculator(user_profile,clustered_articles_matrix) :
    
    user_interested_articles = []
    
    contents_interest_score = []

    user_preffered_articles = cosine_similarity(userProfile_one.reshape(1,-1),clustered_articles_matrix)
    
    top_articles = np.sort(user_preffered_articles).flatten()[::-1][:10]
    
    user_interested_articles.append(top_articles)
    
    content_interest_score = (user_interested_articles[0] * 0.4)
    
    return content_interest_score


content_recommended = Content_Recommends_Calculator(userProfile_one,clustered_articles_matrix)

In [22]:
existing_users = np.random.random_sample(size=(1000,10))   #we take n existing users   
new_user = np.random.random_sample(size=(1,10))            #we take a single new user

In [23]:
def Collaborative_Recommends_Calculator(existing_usr,new_usr) :
    
    collaborative_interest_score = [ ]
    sorted_collaborative_interest = [ ]
    
    collaborative_interest_score = cosine_similarity(existing_users,new_user)
    
    sorted_collaborative_interest = np.argsort(collaborative_interest_score,axis=0)[::-1][:10]
    
    sorted_collaborative_indexes = existing_users[sorted_collaborative_interest]
    
    collab_interest = np.mean(sorted_collaborative_indexes.reshape(-1,10),axis=0)
    
    collaborative_interest_scores = collab_interest*0.6
    
    return collaborative_interest_scores

In [24]:
collab_recommended = Collaborative_Recommends_Calculator(existing_users,new_user)

In [25]:
collab_recommended

array([0.35610965, 0.40705177, 0.13578598, 0.46559692, 0.4294026 ,
       0.10251596, 0.3331978 , 0.46642481, 0.3524417 , 0.13980315])

In [ ]:
hybrid_recommend_indexes =  Hybrid_Calculator()

In [26]:
def Trends(trending) :

    trends = np.mean(existing_users,axis=0)
    Trending_news = cosine_similarity(trends.reshape(1,10),clustered_articles_matrix)
    top= np.sort(Trending_news)[::-1][0][:10]
    return top

In [ ]:
Trending_Articles = Trends(existing_users)*0.3

In [27]:
def Hybrid_Calculator():
    
    hybrid_interests = np.add(content_recommended,collab_recommended)
    
    similar_scores = cosine_similarity(hybrid_interests.reshape(1,10),clustered_articles_matrix)
    
    recommended_article_address =  np.argsort(similar_scores)[::-1]
    
    return recommended_article_address

In [28]:
hybrid_recommend_indexes =  Hybrid_Calculator() #we get hyrid interest with our variations of 0.4 content based and 0.6 collab based

In [29]:
labels = kmeans.labels_

In [30]:
labels

array([5, 4, 7, ..., 8, 9, 9], dtype=int32)

In [32]:
maxx = np.argmax(clustered_articles_matrix,axis=1) 

In [33]:
maxx

array([9, 5, 3, ..., 9, 0, 3])

In [34]:
for articles in hybrid_recommend_indexes :
    
    print('Recommended-Articles :')
    
    print('\n')
    
    print(title[articles][:no_of_recommends])  

Recommended-Articles :


347     Jayasurya s Kuruthakedinte Koodane song from P...
386      Pathemari   5 reasons why Mammootty-Jewel Mar...
1908    Deepika  Salman  Katrina and Other Stars Who E...
129        Kathakali movie review  Live audience response
4601     Vedalam   Vedhalam  box office collection  Aj...
554      Abhishek Bachchan s role in Housefull 3 revealed
42      Vijay 61  Atlee being rewarded even before the...
2035    Maharashtra Imposes Ban on Beef  Traders Upset...
3196    Akshay Kumar in  Holiday 2   will the  Airlift...
130            Gethu movie review  Live audience response
1153    Lenovo A6000 4G Android Smartphone  First Impr...
472      Kyaa Kool Hain Hum 3  banned in Pakistan for ...
246      Baahubali 2  update  Anushka Shetty to take c...
1869    Roundup   My Choice    Tribute to Paul Walker ...
368     Rajamouli s  Baahubali   Mahesh s  Srimanthudu...
134     Madonna Sebastian aka Celine of Premam to star...
2971    Alia Bhatt and Katrina Kaif are the new